# Bibliotecas

In [1]:
import os
import glob
import shutil
import psycopg2
import pandas as pd
from google.colab import drive

# Arquivos do projeto


In [2]:
# Área de trabalho do projeto
drive.mount('/content/drive')
workspace_path = '/content/drive/MyDrive/Workspace'

Mounted at /content/drive


In [3]:
# Caminho para os arquivos dos dados
data_path = f'{workspace_path}/fiap/pos-tech/data-analytics/fase-4/dados'

In [4]:
# Pasta temporária
temp_path = 'tmp'
os.makedirs(temp_path, exist_ok=True)

In [5]:
# Copiando os arquivos do backup do banco de dados
for part_file in glob.glob(f'{data_path}/anp_dump.tar.gz.part-*'):
    shutil.copy(part_file, temp_path)

In [6]:
# Remontando o arquivo de backup no formato .tar.gz
!cat tmp/anp_dump.tar.gz.part-* > tmp/anp_dump.tar.gz

In [7]:
# Descompactando o arquivo de backup do banco de dados
!tar -xzvf tmp/anp_dump.tar.gz

tmp/anp_dump.backup


# PostgreSQL

In [8]:
# Instalação
!apt-get install -y postgresql

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert sysstat
Suggested packages:
  bsd-mailx | mailx postgresql-doc postgresql-doc-14 isag
The following NEW packages will be installed:
  libcommon-sense-perl libjson-perl libjson-xs-perl libtypes-serialiser-perl logrotate netbase
  postgresql postgresql-14 postgresql-client-14 postgresql-client-common postgresql-common ssl-cert
  sysstat
0 upgraded, 13 newly installed, 0 to remove and 49 not upgraded.
Need to get 18.4 MB of archives.
After this operation, 51.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 logrotate amd64 3.19.0-1ubuntu1.1 [54.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd6

In [9]:
# Iniciando o serviço
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [10]:
# Configurando o usuário postgres
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"

ALTER ROLE


In [11]:
# Criando o banco de dados
!sudo -u postgres psql -c "CREATE DATABASE anp;"

CREATE DATABASE


In [12]:
# Restaurando os dados para o PostgreSQL
!PGPASSWORD=postgres pg_restore -U postgres -d anp -h localhost -v tmp/anp_dump.backup

pg_restore: connecting to database for restore
pg_restore: creating TABLE "public.preco_combustivel"
pg_restore: processing data for table "public.preco_combustivel"


# Explorando os dados

In [13]:
# Conexão com o PostgreSQL
conn = psycopg2.connect(
    host='localhost',
    port='5432',
    database='anp',
    user='postgres',
    password='postgres'
)

In [14]:
df = pd.io.sql.read_sql_query('select * from public.preco_combustivel', conn)
df.head()

<ipython-input-14-af937fd07575>:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.io.sql.read_sql_query('select * from public.preco_combustivel', conn)


,regiao_sigla,estado_sigla,municipio,revenda,cnpj_da_revenda,nome_da_rua,numero_rua,complemento,bairro,cep,produto,data_da_coleta,valor_de_venda,unidade_de_medida,bandeira
0,N,AC,RIO BRANCO,AUTO POSTO AMAPA - EIRELI,00.529.581/0001-53,VIA CHICO MENDES,3570,None,AREAL,69906-119,GASOLINA,2022-01-03,6.99,R$ / litro,VIBRA ENERGIA
1,N,AC,RIO BRANCO,AUTO POSTO AMAPA - EIRELI,00.529.581/0001-53,VIA CHICO MENDES,3570,None,AREAL,69906-119,ETANOL,2022-01-03,5.99,R$ / litro,VIBRA ENERGIA
2,N,AC,RIO BRANCO,AUTO POSTO AMAPA - EIRELI,00.529.581/0001-53,VIA CHICO MENDES,3570,None,AREAL,69906-119,DIESEL,2022-01-03,6.09,R$ / litro,VIBRA ENERGIA
3,N,AC,RIO BRANCO,AUTO POSTO AMAPA - EIRELI,00.529.581/0001-53,VIA CHICO MENDES,3570,None,AREAL,69906-119,GASOLINA ADITIVADA,2022-01-03,7.05,R$ / litro,VIBRA ENERGIA
4,N,AC,RIO BRANCO,AUTO POSTO AMAPA - EIRELI,00.529.581/0001-53,VIA CHICO MENDES,3570,None,AREAL,69906-119,DIESEL S10,2022-01-03,6.12,R$ / litro,VIBRA ENERGIA
